In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
corner_track_params = dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)

In [4]:
lk_params = dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [13]:
cap = cv2.VideoCapture(0)

ret,prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

# Points to track
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask = np.zeros_like(prev_frame)
# create a frame if zeros having the same size as prev_frame

while True:
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    nextPts,status,err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
    # Calculate the optical flow tracking method, Pyramid-Lucas Kunade
    good_new = nextPts[status==1]
    # check the flow of features found
    good_prev = prevPts[status==1]
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        x_new , y_new = new.ravel()
        # join the new x and y together
        x_prev , y_prev = prev.ravel()
        # join the old x and y together
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        # add a line to the as the path moved by the edges detected
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
        # draw a circle as the line is moved on the mask
    
    img = cv2.add(frame,mask)
    # add the mask and the image
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(30) & 0xFF 
    if k ==27:
        break
        
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    # update current frame to be previous frame, copy not to overwrite anything
    
cv2.destroyAllWindows()
cap.release()

In [18]:
# Dense Optical Flow
cap = cv2.VideoCapture(0)
ret , frame1 = cap.read()
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255
# Not moving parts are colored as black and moving are color changing
while True:
    ret , frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0)
    # Computes a dense optical flow using the Gunnar Farneback's algorithm
    mag , ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    # convert the cartesian coordinates to polar coordinates
    hsv_mask[:,:,0] = ang/2
    # limit the range for colors
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    # grab channel 2, value, to be stretched to 0 and 255
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    # change color for display
    cv2.imshow('frame',bgr)
    
    k=cv2.waitKey(10) & 0xFF
    if k == 27:
        break
        
    prvsImg = nextImg
    
cap.release()
cv2.destroyAllWindows()